# Generate wind experiments figures

In [27]:
import numpy as np
import xarray as xr

from scores.continuous import mse
from scores.processing import broadcast_and_match_nan
from scores.stats.statistical_tests import diebold_mariano

from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Get Data for wind max-in-hour experiment

In [28]:
official_max_exp = xr.open_dataarray("../data/windmax_exp/Official_WindMag_00_20190901-20191130.nc")
# Note that the 12Z AutoFcst was the automated guidance available to meteorologists for the
# afternoon (00Z) official forecast issue.
autofcst_max_exp = xr.open_dataarray("../data/windmax_exp/FCF_2_0_AutoFcst_WindMag_18_20190901-20191130.nc")
hindcast_max_exp = xr.open_dataarray("../data/windmax_exp/AutoFcstMax_WindMag_12_20190901-20191130.nc")
obs_max_exp = xr.open_dataarray("../data/windmax_exp/obs_WindMagMaxInHour_20190901-20191130.nc")

# Match missing data between datasets
official_max_exp, autofcst_max_exp, hindcast_max_exp, obs_max_exp = broadcast_and_match_nan(official_max_exp, autofcst_max_exp, hindcast_max_exp, obs_max_exp)

### Get data for exposed peaks experiment

In [29]:
# Data for exposed peaks experiment
official_peaks = xr.open_dataarray("../data/exposed_peaks/Official_WindMag_00_20171201-20180228_exposed_peaks.nc")
# The existing automated alternative for the 2017-2018 summer was FCF 1.0 with an 18Z base time
automated_peaks = xr.open_dataarray("../data/exposed_peaks/FCF_1_0_AutoFcst_WindMag_18_20171201-20180228_exposed_peaks.nc")
# The hindcast was called FCF1.3
hindcast_peaks = xr.open_dataarray("../data/exposed_peaks/FCF_1_3_AutoFcst_WindMag_18_20171201-20180228_exposed_peaks.nc")
obs_peaks = xr.open_dataarray("../data/exposed_peaks/obs_WindMag_20171201-20180228_exposed_peaks.nc")

# Match missing data between datasets
official_peaks, automated_peaks, hindcast_peaks, obs_peaks = broadcast_and_match_nan(official_peaks, automated_peaks, hindcast_peaks, obs_peaks)

### Calulate MSE

In [30]:
official_max_exp_mse = mse(official_max_exp, obs_max_exp, preserve_dims="lead_day")
hindcast_max_exp_mse = mse(hindcast_max_exp, obs_max_exp, preserve_dims="lead_day")
autofcst_max_exp_mse = mse(autofcst_max_exp, obs_max_exp, preserve_dims="lead_day")

official_peaks_mse = mse(official_peaks, obs_peaks, preserve_dims="lead_day")
hindcast_peaks_mse = mse(hindcast_peaks, obs_peaks, preserve_dims="lead_day")
autofcst_peaks_mse = mse(automated_peaks, obs_peaks, preserve_dims="lead_day")

In [31]:
official_line_colour = 'rgba(230,159,0,1)'
autofcst_line_colour = 'rgba(86,180,233,1)'
hindcast_line_colour = "#009E73"

figure = make_subplots(rows=1, cols=2, subplot_titles=("<b>(a)</b>", "<b>(b)</b>", ))
figure.update_annotations(font_size=12, xshift=-160, xanchor="left")

# Left subfigure. WindMax experiment
figure.add_trace(
    go.Scatter(
        x=official_max_exp_mse.lead_day,
        y=official_max_exp_mse.values,
        line=dict(color=official_line_colour),
        name="Official"
    ), row=1, col=1
)
figure.add_trace(
    go.Scatter(
        x=autofcst_max_exp_mse.lead_day,
        y=autofcst_max_exp_mse.values,
        line=dict(color=autofcst_line_colour),
        name="Existing automated"
    ), row=1, col=1
)
figure.add_trace(
    go.Scatter(
        x=hindcast_max_exp_mse.lead_day,
        y=hindcast_max_exp_mse.values,
        line=dict(color=hindcast_line_colour),
        name="Hindcast experiment"
    ), row=1, col=1
)

# Right subfigure. Wind exposed peaks experiment
figure.add_trace(
    go.Scatter(
        x=official_peaks_mse.lead_day,
        y=official_peaks_mse.values,
        line=dict(color=official_line_colour),
        name="Official",
        showlegend=False
    ), row=1, col=2
)
figure.add_trace(
    go.Scatter(
        x=autofcst_peaks_mse.lead_day,
        y=autofcst_peaks_mse.values,
        line=dict(color=autofcst_line_colour),
        name="Existing Automated",
        showlegend=False
    ), row=1, col=2
)
figure.add_trace(
    go.Scatter(
        x=hindcast_peaks_mse.lead_day,
        y=hindcast_peaks_mse.values,
        line=dict(color=hindcast_line_colour),
        name="New Automated",
        showlegend=False
    ), row=1, col=2
)

figure.update_layout(
    legend=dict(x=0.01,y=0.99),
    height=400,
    width=800,
    margin=go.layout.Margin(
        l=20, #left margin
        r=20, #right margin
        b=20, #bottom margin
        t=20, #top margin
    ),
)
figure.update_yaxes(title_text="MSE (kt<sup>2</sup>)", row=1, col=1)
figure.update_yaxes(title_text="MSE (kt<sup>2</sup>)", row=1, col=2)

figure.update_xaxes(title_text="Lead day", row=1, col=1, tickmode="linear", tick0=0, dtick=1)
figure.update_xaxes(title_text="Lead day", row=1, col=2, tickmode="linear", tick0=0, dtick=1)

In [32]:
figure.write_image("../figures/wind_exp.svg")

### Statistical significance testing for wind max experiment

In [33]:
official_max_exp_mse = mse(official_max_exp, obs_max_exp, preserve_dims=["lead_day", "valid_start"])
hindcast_max_exp_mse = mse(hindcast_max_exp, obs_max_exp, preserve_dims=["lead_day", "valid_start"])
autofcst_max_exp_mse = mse(autofcst_max_exp, obs_max_exp, preserve_dims=["lead_day", "valid_start"])

In [34]:
# Difference between Official and the hindcast
diff_official_hindcast = official_max_exp_mse - hindcast_max_exp_mse
diff_official_hindcast = diff_official_hindcast.assign_coords(h=("lead_day", [2, 3, 4, 5, 6, 7, 8]))
dm_result = diebold_mariano(diff_official_hindcast, "lead_day", "h")
dm_result

<xarray.Dataset>
Dimensions:          (lead_day: 7)
Coordinates:
  * lead_day         (lead_day) int64 1 2 3 4 5 6 7
Data variables:
    mean             (lead_day) float64 2.561 3.068 1.978 ... 2.437 2.323 2.492
    dm_test_stat     (lead_day) float64 18.58 19.77 13.85 ... 9.899 7.875 5.959
    timeseries_len   (lead_day) int64 2184 2184 2184 2184 2184 2184 2184
    confidence_gt_0  (lead_day) float64 1.0 1.0 1.0 1.0 1.0 1.0 1.0
    ci_upper         (lead_day) float64 2.831 3.372 2.257 ... 2.919 2.902 3.312
    ci_lower         (lead_day) float64 2.291 2.764 1.698 ... 1.954 1.745 1.673

In [35]:
# Difference between the existing AutoFcst and the hindcast
diff_autofcst_hindcast = autofcst_max_exp_mse - hindcast_max_exp_mse
diff_autofcst_hindcast = diff_autofcst_hindcast.assign_coords(h=("lead_day", [2, 3, 4, 5, 6, 7, 8]))
dm_result = diebold_mariano(diff_autofcst_hindcast, "lead_day", "h")
dm_result

<xarray.Dataset>
Dimensions:          (lead_day: 7)
Coordinates:
  * lead_day         (lead_day) int64 1 2 3 4 5 6 7
Data variables:
    mean             (lead_day) float64 2.841 2.846 2.869 ... 2.915 2.804 2.708
    dm_test_stat     (lead_day) float64 21.36 20.26 18.45 ... 10.99 8.707 6.401
    timeseries_len   (lead_day) int64 2184 2184 2184 2184 2184 2184 2184
    confidence_gt_0  (lead_day) float64 1.0 1.0 1.0 1.0 1.0 1.0 1.0
    ci_upper         (lead_day) float64 3.102 3.121 3.173 ... 3.435 3.436 3.537
    ci_lower         (lead_day) float64 2.581 2.57 2.564 2.49 2.395 2.173 1.879

In [36]:
# Difference between the existing AutoFcst and the Official
diff_autofcst_official = autofcst_max_exp_mse - official_max_exp_mse
diff_autofcst_official = diff_autofcst_official.assign_coords(h=("lead_day", [2, 3, 4, 5, 6, 7, 8]))
dm_result = diebold_mariano(diff_autofcst_official, "lead_day", "h")
dm_result

<xarray.Dataset>
Dimensions:          (lead_day: 7)
Coordinates:
  * lead_day         (lead_day) int64 1 2 3 4 5 6 7
Data variables:
    mean             (lead_day) float64 0.2801 -0.2222 0.891 ... 0.4809 0.2153
    dm_test_stat     (lead_day) float64 2.11 -1.432 8.356 ... 4.776 3.862 1.837
    timeseries_len   (lead_day) int64 2184 2184 2184 2184 2184 2184 2184
    confidence_gt_0  (lead_day) float64 0.9826 0.07611 1.0 1.0 1.0 0.9999 0.9669
    ci_upper         (lead_day) float64 0.5403 0.08197 1.1 ... 0.725 0.445
    ci_lower         (lead_day) float64 0.01992 -0.5263 ... 0.2369 -0.01444

### Statistical significance testing for exposed peaks experiment

In [37]:
official_peaks_mse = mse(official_peaks, obs_peaks, preserve_dims=["lead_day", "valid_start"])
hindcast_peaks_mse = mse(hindcast_peaks, obs_peaks, preserve_dims=["lead_day", "valid_start"])
autofcst_peaks_mse = mse(automated_peaks, obs_peaks, preserve_dims=["lead_day", "valid_start"])

In [38]:
# Difference between Official and the hindcast
diff_autofcst_hindcast = autofcst_peaks_mse - hindcast_peaks_mse
diff_autofcst_hindcast = diff_autofcst_hindcast.assign_coords(h=("lead_day", [2, 3, 4, 5, 6, 7, 8]))
dm_result = diebold_mariano(diff_autofcst_hindcast, "lead_day", "h")
dm_result

/home/nloveday/scores/src/scores/stats/statistical_tests/diebold_mariano_impl.py:251: RuntimeWarning:

A least one NaN value was detected in `da_timeseries`. This may impact the calculation of autocovariances.



<xarray.Dataset>
Dimensions:          (lead_day: 7)
Coordinates:
  * lead_day         (lead_day) int64 1 2 3 4 5 6 7
Data variables:
    mean             (lead_day) float64 42.41 41.42 46.73 ... 42.64 39.96 32.96
    dm_test_stat     (lead_day) float64 12.68 12.51 10.82 ... 9.319 9.542 8.867
    timeseries_len   (lead_day) int64 2160 2160 2160 2160 2160 2160 2068
    confidence_gt_0  (lead_day) float64 1.0 1.0 1.0 1.0 1.0 1.0 1.0
    ci_upper         (lead_day) float64 48.97 47.9 55.2 52.89 51.61 48.17 40.25
    ci_lower         (lead_day) float64 35.86 34.93 38.26 ... 33.68 31.75 25.68